# Setup

In [1]:
from langgraph.graph import StateGraph
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnableLambda
from langchain.prompts import ChatPromptTemplate
from duckduckgo_search import DDGS
from typing import TypedDict
import os
import yaml

In [2]:
with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)
os.environ["GOOGLE_API_KEY"] = config["GOOGLE_API_KEY"]

# LLM

In [3]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

# State

In [4]:
class QAState(TypedDict):
    question: str
    answer: str

# Callback Functions

In [5]:
def receive_question(state: QAState) -> QAState:
    return state

def web_search(state: QAState) -> QAState:
    question = state["question"]

    with DDGS() as ddgs:
        results = ddgs.text(question, max_results=1)

    context = "\n".join([r["body"] for r in results if "body" in r]) if results else "Nenhum resultado encontrado."

    prompt = ChatPromptTemplate.from_template(
    """Use o seguinte contexto para responder à pergunta:
    Contexto: {context}
    Pergunta: {question}
    Resposta:"""
    )

    chain = prompt | llm
    answer = chain.invoke({"context": context, "question": question})

    return {"question": question, "answer": answer.content}

def model_answer(state: QAState) -> QAState:
    question = state["question"]
    prompt = ChatPromptTemplate.from_template("Responda à seguinte pergunta: {question}")
    chain = prompt | llm
    answer = chain.invoke({"question": state["question"]})
    return {"question": question, "answer": answer.content}

def show_result(state: QAState) -> QAState:
    print("Resposta final:")
    print(state["answer"])
    return state

In [6]:
def decide_route(state: QAState) -> QAState:
    question = state["question"].lower()
    next_step = "web" if any(word in question for word in ["hoje", "agora", "notícia", "notícias"]) else "llm"
    return {**state, "next_step": next_step}


# Graph

In [7]:
builder = StateGraph(QAState)

# Nodes

In [8]:
builder.add_node("receive_question", RunnableLambda(receive_question))
builder.add_node("decide_route", RunnableLambda(decide_route))
builder.add_node("web_search", RunnableLambda(web_search))
builder.add_node("model_answer", RunnableLambda(model_answer))
builder.add_node("show_result", RunnableLambda(show_result))

# Edges & State Conditions

In [9]:
builder.set_entry_point("receive_question")
builder.add_edge("receive_question", "decide_route")
builder.add_conditional_edges(
    "decide_route",
    lambda state: state["next_step"],
    {
        "web": "web_search",
        "llm": "model_answer"
    }
)
builder.add_edge("web_search", "show_result")
builder.add_edge("model_answer", "show_result")
builder.set_finish_point("show_result")

# Execution

In [10]:
graph = builder.compile()

print("[TEST 1]")
graph.invoke({"question": "Qual a capital do Brasil?"})
print("\n[TEST 2]")
graph.invoke({"question": "Quais as últimas notícias do brasil?"})

[TEST 1]
Resposta final:
A capital do Brasil é **Brasília**.

[TEST 2]
Resposta final:
A resposta, baseada no contexto fornecido, seria:

"As últimas notícias do Brasil abrangem política, economia, emprego, educação, saúde, meio ambiente, tecnologia, ciência, cultura e carros. Para informações mais detalhadas, você pode assistir aos vídeos dos telejornais da TV Globo e da GloboNews."


{'question': 'Quais as últimas notícias do brasil?',
 'answer': 'A resposta, baseada no contexto fornecido, seria:\n\n"As últimas notícias do Brasil abrangem política, economia, emprego, educação, saúde, meio ambiente, tecnologia, ciência, cultura e carros. Para informações mais detalhadas, você pode assistir aos vídeos dos telejornais da TV Globo e da GloboNews."'}

# Completions

In [11]:
dot = graph.get_graph().draw_mermaid()

print(dot)

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	receive_question(receive_question)
	decide_route(decide_route)
	web_search(web_search)
	model_answer(model_answer)
	show_result(show_result)
	__end__([<p>__end__</p>]):::last
	__start__ --> receive_question;
	decide_route -. &nbsp;llm&nbsp; .-> model_answer;
	decide_route -. &nbsp;web&nbsp; .-> web_search;
	model_answer --> show_result;
	receive_question --> decide_route;
	web_search --> show_result;
	show_result --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

